# Sample Notebook

1. Setup Google Cloud service account
2. Download JSON auth keys and upload to the files
2. Share Google Drive folder with service account

### Setting up codebase and DVC

In [ ]:
%%capture
!pip install dvc[gdrive] mlflow pyyaml
!git clone https://github.com/pavelihno/colab-dvc-template.git

In [2]:
%cd colab-dvc-template/

/content/colab-dvc-template


In [6]:
import os

GDRIVE_FOLDER_ID = '<FOLDER_ID>'
SERVICE_ACCOUNT_KEY_PATH = '/content/service-key.json'

os.environ['GDRIVE_FOLDER_ID'] = GDRIVE_FOLDER_ID
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_KEY_PATH

In [7]:
%%capture
!dvc remote add -d -f gdrive "gdrive://$GDRIVE_FOLDER_ID"
!dvc remote modify gdrive gdrive_use_service_account true
!dvc remote modify gdrive gdrive_service_account_json_file_path "$GOOGLE_APPLICATION_CREDENTIALS"
!dvc pull

### Import libraries

In [ ]:
import tensorflow as tf

from src.train import train_model
from src.eval import evaluate_model
from src.utils import load_config, generate_sample_data
from src.model.sample import SampleNeuralNetwork

### Training and evaluating model

In [ ]:
config = load_config('configs/default.yaml')

run_id, model_path = train_model(config)

print(f'\nTraining completed!')
print(f'Run ID: {run_id}')
print(f'Model saved to: {model_path}')

In [ ]:
# Evaluate using the run_id from training
metrics = evaluate_model(config, run_id=run_id)

print(f'\nEvaluation Results:')
print(f'Test Loss: {metrics["test_loss"]:.4f}')
print(f'Test Accuracy: {metrics["test_accuracy"]:.4f}')

### Push MLflow artifacts to DVC/Google Drive

In [ ]:
!dvc add mlruns

!dvc add data/*.npy
!dvc add models/*.keras

!git add mlruns.dvc data/*.dvc models/*.dvc .gitignore
!git commit -m "Update MLflow experiments, data, and models"

!dvc push

print("MLflow artifacts synced to Google Drive!")